# Analiza Skuteczności Kampanii Promocyjnej

W tym notebooku przeprowadzamy analizę skuteczności kampanii promocyjnej na podstawie danych z pliku `kampania.csv` oraz opisu cech z `kampania - cechy.txt`.

**Cele:**
- Wczytać i przygotować dane (oczyszczenie, kodowanie zmiennych, przekształcenie daty).
- Podzielić dane na zbiór treningowy i testowy.
- Zbudować modele klasyfikacyjne: KNN oraz SVM.
- Porównać wyniki dla danych oryginalnych i danych znormalizowanych.


In [3]:
import pandas as pd
import numpy as np

# Klasyfikatory
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Skalowanie
from sklearn.preprocessing import StandardScaler

# Podział na train/test i metryki
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score

Wczytanie danych

In [12]:
df = pd.read_csv('kampania.csv', sep='\t')

Wstępna analiza i obróbka danych

In [14]:
# Podstawowy podgląd informacji o danych
print("Informacje o danych:")
print(df.info())

# Sprawdzenie brakujących wartości
print("\nLiczba brakujących wartości:")
print(df.isnull().sum())

# Jeśli braki są niewielkie, usuwamy wiersze z brakami
df.dropna(inplace=True)

# Konwersja kolumny 'Dt_Customer' do typu datetime oraz obliczenie stażu klienta
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)
max_date = df['Dt_Customer'].max()  # ostatni dzień w zbiorze
df['CustomerTenure'] = (max_date - df['Dt_Customer']).dt.days
df.drop(columns=['Dt_Customer'], inplace=True)

# Kodowanie zmiennych kategorycznych - Education i Marital_Status
df = pd.get_dummies(df, columns=['Education', 'Marital_Status'], drop_first=True)

# Podgląd przetworzonych danych
print(df.head())


Informacje o danych:
<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2216 non-null   int64  
 1   Year_Birth           2216 non-null   int64  
 2   Education            2216 non-null   object 
 3   Marital_Status       2216 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2216 non-null   int64  
 6   Teenhome             2216 non-null   int64  
 7   Dt_Customer          2216 non-null   object 
 8   Recency              2216 non-null   int64  
 9   MntWines             2216 non-null   int64  
 10  MntFruits            2216 non-null   int64  
 11  MntMeatProducts      2216 non-null   int64  
 12  MntFishProducts      2216 non-null   int64  
 13  MntSweetProducts     2216 non-null   int64  
 14  MntGoldProds         2216 non-null   int64  
 15  NumDealsPurchases    2

Przygotowanie zbioru cech (X) i etykiety (y) oraz podział na treningowy i testowy

In [15]:
# Wykluczamy kolumnę 'ID' (niepotrzebna) oraz 'Response' jest zmienną docelową
X = df.drop(columns=['ID', 'Response'])
y = df['Response']

# Podział na zbiór treningowy i testowy (80%/20%) z zachowaniem proporcji klas
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Rozmiar zbioru treningowego:", X_train.shape)
print("Rozmiar zbioru testowego:", X_test.shape)


Rozmiar zbioru treningowego: (1772, 36)
Rozmiar zbioru testowego: (444, 36)


Budowa modeli na danych oryginalnych

In [16]:
# 1. Model KNN
knn_orig = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
knn_orig.fit(X_train, y_train)
y_pred_knn_orig = knn_orig.predict(X_test)

acc_knn_orig = accuracy_score(y_test, y_pred_knn_orig)
f1_knn_orig = f1_score(y_test, y_pred_knn_orig)

print(f"KNN (oryginalne) - Accuracy: {acc_knn_orig:.3f}, F1: {f1_knn_orig:.3f}")

# 2. Model SVM
svm_orig = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_orig.fit(X_train, y_train)
y_pred_svm_orig = svm_orig.predict(X_test)

acc_svm_orig = accuracy_score(y_test, y_pred_svm_orig)
f1_svm_orig = f1_score(y_test, y_pred_svm_orig)

print(f"SVM (oryginalne) - Accuracy: {acc_svm_orig:.3f}, F1: {f1_svm_orig:.3f}")


KNN (oryginalne) - Accuracy: 0.831, F1: 0.176
SVM (oryginalne) - Accuracy: 0.849, F1: 0.000


Budowa modeli na danych znormalizowanych

In [17]:
# Skalowanie danych (uczestwujemy tylko zbiór treningowy, a następnie stosujemy transformację na zbiorze testowym)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Model KNN na danych znormalizowanych
knn_scaled = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
knn_scaled.fit(X_train_scaled, y_train)
y_pred_knn_scaled = knn_scaled.predict(X_test_scaled)

acc_knn_scaled = accuracy_score(y_test, y_pred_knn_scaled)
f1_knn_scaled = f1_score(y_test, y_pred_knn_scaled)

print(f"KNN (znormalizowane) - Accuracy: {acc_knn_scaled:.3f}, F1: {f1_knn_scaled:.3f}")

# 2. Model SVM na danych znormalizowanych
svm_scaled = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_scaled.fit(X_train_scaled, y_train)
y_pred_svm_scaled = svm_scaled.predict(X_test_scaled)

acc_svm_scaled = accuracy_score(y_test, y_pred_svm_scaled)
f1_svm_scaled = f1_score(y_test, y_pred_svm_scaled)

print(f"SVM (znormalizowane) - Accuracy: {acc_svm_scaled:.3f}, F1: {f1_svm_scaled:.3f}")


KNN (znormalizowane) - Accuracy: 0.881, F1: 0.454
SVM (znormalizowane) - Accuracy: 0.890, F1: 0.473


Podsumowanie wyników

In [18]:
print("=== Podsumowanie Wyników ===")
print(f"KNN (oryginalne): Accuracy = {acc_knn_orig:.3f}, F1 = {f1_knn_orig:.3f}")
print(f"KNN (znormalizowane): Accuracy = {acc_knn_scaled:.3f}, F1 = {f1_knn_scaled:.3f}")
print(f"SVM (oryginalne): Accuracy = {acc_svm_orig:.3f}, F1 = {f1_svm_orig:.3f}")
print(f"SVM (znormalizowane): Accuracy = {acc_svm_scaled:.3f}, F1 = {f1_svm_scaled:.3f}")


=== Podsumowanie Wyników ===
KNN (oryginalne): Accuracy = 0.831, F1 = 0.176
KNN (znormalizowane): Accuracy = 0.881, F1 = 0.454
SVM (oryginalne): Accuracy = 0.849, F1 = 0.000
SVM (znormalizowane): Accuracy = 0.890, F1 = 0.473
